## The Andong Kwŏn Genealogy of 1476: Neo4j Bulk-Import Notebook

This notebook bulk imports the *Andong Kwŏn Genealogy of 1476* (*Andong Kwŏn ssi Sŏnghwabo* 安東權氏成化譜) node and edge tables into a Neo4j database, which enables the exexcution of robust graph queries and machine-assisted exploration of this valuable historical source.

---

### Overview of notebook functions

1. **Load source files**  
   * `tsv/andongkwon_1476_nodes.tsv` (node attribute table)  
   * `tsv/andongkwon_1476_edges.tsv` (edge list)

2. **Unicode normalization**  
   All string fields are normalized using **NFKC** to eliminate hidden duplicates caused by visually similar characters mapped onto distinct Unicode values.

3. **Index creation**  
   Adds indexes to optimize graph traversal and improve performance during data import.

---

### Prerequisites

The notebook requires an active Neo4j instance:

1. Install Neo4j Desktop or Neo4j Server (community or enterprise edition works equally well).
2. Edit the default address `bolt://localhost:7687` according to your configuration, if necessary.
3. Check the default username `neo4j` and insert your own password in the code cell that initializes the driver. Never commit this credential to a public repository.
4. The TSV files are read directly from the `tsv/` folder in this GitHub repository.

---

### Notes

* See `Andong Kwon Genealogy Inspection.ipynb` for page-by-page visualizations of the genealogy.

In [1]:
from neo4j import GraphDatabase

In [2]:
#driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", ""))

In [4]:
# Session 1: Delete all existing nodes and relationships.
with driver.session() as session:
    result = session.run("""
        MATCH (m) DETACH DELETE (m)
    """)
    print(f"Deleted nodes: {result.consume().counters.nodes_deleted}.")

# Session 2: Load nodes from TSV.
with driver.session() as session:
    result = session.run("""
        LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/javiercha/andong-kwon-1476/refs/heads/main/tsv/andongkwon_1476_nodes.tsv' as row FIELDTERMINATOR '\t'
        CREATE (m:Person) set m+=row
        RETURN count(m)
    """)
    for record in result:
        print(f"Created nodes: {record[0]}.")

# Sessions 3 and 4: Create indexes on biog_id and choronym.
with driver.session() as session:
    result = session.run("""
        CREATE INDEX IF NOT EXISTS FOR (m:Person) ON (m.biog_id)
    """)
with driver.session() as session:
    result = session.run("""
        CREATE INDEX IF NOT EXISTS FOR (m:Person) ON (m.biog_id)
    """)

# Session 5: Load edges from TSV and create relationships.
with driver.session() as session:
    result = session.run("""
        LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/javiercha/andong-kwon-1476/refs/heads/main/tsv/andongkwon_1476_edges.tsv' AS row FIELDTERMINATOR '\t'
        MERGE (source:Person {biog_id: row.source})
          ON CREATE SET source.name = row.source_name,
                        source.vol = toInteger(row.vol),
                        source.leaf = toInteger(row.leaf),
                        source.side = row.side,
                        source.page = row.page,
                        source.page2024 = row.page2024,
                        source.reference = row.reference
        MERGE (target:Person {biog_id: row.target})
          ON CREATE SET target.name = row.target_name,
                        target.vol = toInteger(row.vol),
                        target.leaf = toInteger(row.leaf),
                        target.side = row.side,
                        target.page = row.page,
                        target.page2024 = row.page2024,
                        target.reference = row.reference
        WITH source, target, row,
             CASE
                 WHEN source.choronym = '安東 權' AND target.choronym = '安東 權' AND row.type = 'HAS_SON' THEN 'HAS_SON_AK'
                 ELSE row.type + '_SSB'
             END as relationshipType
        CALL apoc.create.relationship(source, relationshipType, {
            child_order: toInteger(row.child_order),
            wife_note: row.wife_note,
            reference: row.reference,
            vol: toInteger(row.vol),
            leaf: toInteger(row.leaf),
            side: row.side
        }, target) YIELD rel
        RETURN count(rel)
    """)
    for record in result:
        print(f"Created relationships: {record[0]}.")

# Session 6: Merge duplicate nodes.
with driver.session() as session:
    result = session.run("""
        MATCH (n:Person)
        WITH n.biog_id AS biogId, collect(n) AS nodesToMerge
        WHERE size(nodesToMerge) > 1
        CALL apoc.refactor.mergeNodes(nodesToMerge, {
            properties: 'combine'
        }) YIELD node
        RETURN count(node) AS mergedNodesCount
    """)
    for record in result:
        print(f"Merged nodes: {record[0]}.")

# Session 7: Merge duplicate relationships.
with driver.session() as session:
    result = session.run("""
        MATCH (source)-[r]->(target)
        WITH source, target, collect(r) AS rels, type(r) AS relType, COUNT(r) AS count
        WHERE count > 1
        CALL apoc.refactor.mergeRelationships(rels, {mergeRels: true}) YIELD rel
        RETURN count(rel) AS mergedRelationshipCount
    """)
    for record in result:
        print(f"Merged relationships: {record[0]}.")
        
# Session 8: Update missing choronym data in HAS_SON chains up to 5 node distances.
with driver.session() as session:
    result = session.run(""" 
        MATCH path = (n)-[r*1..5]-(m)
        WHERE all(rel IN relationships(path) WHERE type(rel) STARTS WITH 'HAS_SON')
          AND any(x IN nodes(path) WHERE x.choronym IS NOT NULL)
          AND any(x IN nodes(path) WHERE x.choronym IS NULL)

        WITH
             nodes(path)                                                  AS chain,
             [x IN nodes(path) WHERE x.choronym IS NOT NULL]              AS filled,
             [x IN nodes(path) WHERE x.choronym IS NULL]                  AS empty

        WITH
             chain, empty,
             apoc.coll.toSet(
                 apoc.coll.flatten(
                     [x IN filled | x.choronym]
                 )
             ) AS uniqChoronym

        UNWIND empty AS node
        SET   node.choronym = uniqChoronym

        WITH chain, node
        RETURN count(DISTINCT chain) AS updatedChainCount,
               count(node)           AS updatedNodeCount;
    """)
    for record in result:
        print(f"Updated chains: {record['updatedChainCount']}.")
        print(f"Updated nodes: {record['updatedNodeCount']}.")

# Session 10: If a choronym is a list with only one entry, convert to string.
with driver.session() as session:
    result = session.run("""
        MATCH (n)
        WHERE n.choronym IS NOT NULL
          AND apoc.convert.toJson(n.choronym) STARTS WITH '['
          AND size(n.choronym) = 1
        WITH n, n.choronym[0] AS newValue       // extract the lone entry
        SET  n.choronym = newValue              // list → scalar
        RETURN count(n) AS conversions
    """).single()["conversions"]

print(f"{result} choronym value(s) converted from one-item lists to strings.")

Deleted nodes: 7580.
Created nodes: 7424.
Created relationships: 8703.
Merged nodes: 3.
Merged relationships: 62.
Updated chains: 9954.
Updated nodes: 12014.
169 choronym value(s) converted from one-item lists to strings.
